In [1]:
import cv2
import os
from glob import glob
from tqdm.notebook import tqdm
import json
from random import sample
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import torch

In [2]:
# 이미지 받고, w, h 확인
# 바운딩 박스 정보 받기
# 긴 쪽 길이 확인
# 긴 쪽 10% 더 붙이고 다른쪽 비율 확인
# 옮기면서 데이터 셋 만들기

# 겹쳐지지 않는 영역에서 이미지 뽑기

# 정상 이미지 데이터, 불량 이미지 데이터 분류
# 사이즈 224로 변경
# 모델 로딩
# 학습
# 적중률 확인

# 불량 이미지 셋은 따로 저장
# 이후에 세그멘 테이션 적용


In [3]:
SIZE = 224
CHOISE_NUM = 10
MARGIN = 10
datasets = list()
PATH_ALL = ['../선박·해양플랜트/모재/불량품/','../자동차/도어/불량품/','../자동차/휀더/불량품/','../자동차/범퍼/불량품/']
PATH_OK_ALL = ['../선박·해양플랜트/모재/양품/','../자동차/도어/양품/','../자동차/휀더/양품/','../자동차/범퍼/양품/']
for PATH, PATH_OK in zip(PATH_ALL, PATH_OK_ALL):
    file_list = glob(PATH+'*.*')
    file_list_ok = glob(PATH_OK+'*.jpg')+glob(PATH_OK+'*.jpeg')+glob(PATH_OK+'*.JPG')+glob(PATH_OK+'*.JPEG')
    for file_path in tqdm(file_list):
        file_name = file_path.split('/')[-1]
        if file_name.split('.')[-1].lower() in ['jpg', 'jpeg', 'png']:
            json_name = file_name.split('.')[0]+".json"
            with open(PATH+json_name, "r",encoding='utf-8-sig') as f:
                tmp = json.load(f)
            WIDTH = tmp['images'][0]['width']
            HEIGHT = tmp['images'][0]['height']
            img = cv2.imread(file_path)
            HEIGHT, WIDTH, CHANNEL = img.shape
            count = 0
            FLAG = False
            while True:
                file_ok_path = random.choice(file_list_ok)
                img_ok = cv2.imread(file_ok_path)
                if img.shape == img_ok.shape:
                    break
                count +=1
                if count>10:
                    FLAG = True
                    break
            if FLAG:continue
            annotations_data = tmp['annotations']
            if len(annotations_data) ==0: continue
            if 'segmentation' not in annotations_data[0].keys(): continue
            for annotation in annotations_data:
                pts=list()
                h, w, c = img.shape
                mask = np.zeros((h,w), dtype=np.uint8)
                x, y, w, h = np.array(annotation['bbox']).astype(int)
                for i in range(len(annotation['segmentation'])//2):
                    xx = annotation['segmentation'][2*i]
                    yy = annotation['segmentation'][2*i+1]
                    pts.append((int(xx),int(yy)))
                pts = np.array(pts, dtype=np.int32)
                cv2.fillPoly(mask, [pts], (255,255,255),cv2.LINE_8)
                # pickup no image
                if w>=h:
                    if x>=10 and WIDTH-x>=10:
                        start = y+HEIGHT-(WIDTH+20) if y+HEIGHT-(WIDTH+20)>0 else 0
                        end = y+(WIDTH+20) if y+(WIDTH+20) < HEIGHT else HEIGHT
                        tmp_area = [i for i in range(end-start-h)]
                        if len(tmp_area)>=CHOISE_NUM:
                            output = sample(tmp_area,10)
                            for sample_num in output:
                                tmp_img = img[sample_num:sample_num+(w+20),x-10:x+w+10] # WIDTH+20
                                tmp_ok_img = img_ok[sample_num:sample_num+(w+20),x-10:x+w+10] # WIDTH+20
                                tmp_mask = mask[sample_num:sample_num+(w+20),x-10:x+w+10] # WIDTH+20
                                if np.sum(tmp_mask)<1e5:continue

                                tmp_img=cv2.resize(tmp_img,(224,224))
                                tmp_ok_img=cv2.resize(tmp_ok_img,(224,224))
                                tmp_mask=cv2.resize(tmp_mask,(224,224))
                                
                                datasets.append((tmp_img,1, tmp_mask)) # 1: 불량
                                datasets.append((tmp_ok_img,0, None)) # 1: 불량
                                
                    if x<10:
                        start = y-(WIDTH) if y-(WIDTH)>0 else 0
                        end = y+(WIDTH) if y+(WIDTH) < HEIGHT else HEIGHT
                        tmp_area = [i for i in range(end-start-h+1)]
                        if len(tmp_area)>=CHOISE_NUM:
                            output = sample(tmp_area,10)
                            for sample_num in output:
                                tmp_img = img[sample_num:sample_num+w,0:w] # WIDTH+20
                                tmp_ok_img = img_ok[sample_num:sample_num+w,0:w] # WIDTH+20
                                tmp_mask = mask[sample_num:sample_num+w,0:w] # WIDTH+20
                                if np.sum(tmp_mask)<1e5:continue       
                                tmp_img=cv2.resize(tmp_img,(224,224))
                                tmp_ok_img=cv2.resize(tmp_ok_img,(224,224))
                                tmp_mask=cv2.resize(tmp_mask,(224,224))                         
                                datasets.append((tmp_img,1,tmp_mask)) # 1: 불량
                                datasets.append((tmp_ok_img,0,None)) # 1: 불량
                        else:
                            for sample_num in tmp_area:
                                tmp_img = img[sample_num:sample_num+w,0:w] # WIDTH+20
                                tmp_ok_img = img_ok[sample_num:sample_num+w,0:w] # WIDTH+20
                                tmp_mask = mask[sample_num:sample_num+w,0:w] # WIDTH+20
                                if np.sum(tmp_mask)<1e5:continue       
                                tmp_img=cv2.resize(tmp_img,(224,224))
                                tmp_ok_img=cv2.resize(tmp_ok_img,(224,224))
                                tmp_mask=cv2.resize(tmp_mask,(224,224))                         
                                datasets.append((tmp_img,1, tmp_mask)) # 1: 불량
                                datasets.append((tmp_ok_img,0,None)) # 1: 불량
                elif w<h:
                    if y>=10 and WIDTH-y>=10:
                        start = x+WIDTH-(HEIGHT+20) if x+WIDTH-(HEIGHT+20)>0 else 0
                        end = x+(HEIGHT+20) if x+(HEIGHT+20) < WIDTH else WIDTH
                        tmp_area = [i for i in range(end-start-w)]
                        if len(tmp_area)>=CHOISE_NUM:
                            output = sample(tmp_area,10)
                            for sample_num in output:
                                tmp_img = img[y-10:y+h+10,sample_num:sample_num+(h+20)] # WIDTH+20
                                tmp_ok_img = img_ok[y-10:y+h+10,sample_num:sample_num+(h+20)] # WIDTH+20
                                tmp_mask = mask[y-10:y+h+10,sample_num:sample_num+(h+20)] # WIDTH+20
                                if np.sum(tmp_mask)<1e5:continue
                                tmp_img=cv2.resize(tmp_img,(224,224))
                                tmp_ok_img=cv2.resize(tmp_ok_img,(224,224))
                                tmp_mask=cv2.resize(tmp_mask,(224,224))
                                datasets.append((tmp_img,1, tmp_mask)) # 1: 불량
                                datasets.append((tmp_ok_img,0, None)) # 0: 정상
                                
                    if y<10:
                        start = x-(HEIGHT) if x-(HEIGHT)>0 else 0
                        end = x+(HEIGHT) if x+(HEIGHT) < WIDTH else WIDTH
                        tmp_area = [i for i in range(end-start-w+1)]
                        if len(tmp_area)>=CHOISE_NUM:
                            output = sample(tmp_area,10)
                            for sample_num in output:
                                tmp_img = img[0:h,sample_num:sample_num+h] # WIDTH+20
                                tmp_ok_img = img_ok[0:h,sample_num:sample_num+h] # WIDTH+20
                                tmp_mask = mask[0:h,sample_num:sample_num+h] # WIDTH+20
                                if np.sum(tmp_mask)<1e5:continue       
                                tmp_img=cv2.resize(tmp_img,(224,224))
                                tmp_ok_img=cv2.resize(tmp_ok_img,(224,224))
                                tmp_mask=cv2.resize(tmp_mask,(224,224))                         
                                datasets.append((tmp_img,1,tmp_mask)) # 1: 불량
                                datasets.append((tmp_ok_img,0,None)) # 1: 불량
                        else:
                            for sample_num in tmp_area:
                                tmp_img = img[0:h,sample_num:sample_num+h] # WIDTH+20
                                tmp_ok_img = img_ok[0:h,sample_num:sample_num+h] # WIDTH+20
                                tmp_mask = mask[0:h,sample_num:sample_num+h] # WIDTH+20
                                if np.sum(tmp_mask)<1e5:continue       
                                tmp_img=cv2.resize(tmp_img,(224,224))
                                tmp_ok_img=cv2.resize(tmp_ok_img,(224,224))
                                tmp_mask=cv2.resize(tmp_mask,(224,224))                         
                                datasets.append((tmp_img,1, tmp_mask)) # 1: 불량
                                datasets.append((tmp_ok_img,0,None)) # 1: 불량
                    

  0%|          | 0/1670 [00:00<?, ?it/s]

  0%|          | 0/1290 [00:00<?, ?it/s]

  0%|          | 0/1944 [00:00<?, ?it/s]

  0%|          | 0/1239 [00:00<?, ?it/s]

In [4]:
classification_datasets = list()
for dataset in datasets:
    img, label, mask = dataset
    h, w, c = img.shape
    img = img.reshape(c,h,w)
    classification_datasets.append((img/255.,label))
with open('detected_img_classification_datasets.pickle', 'wb') as f:
    pickle.dump(classification_datasets,f)

In [6]:
segmentation_datasets = list()
for dataset in datasets:
    img, label, mask = dataset
    h, w, c = img.shape
    img = img.reshape(c,h,w)
    segmentation_datasets.append((img/255.,mask))
with open('detected_img_segmentation_datasets.pickle', 'wb') as f:
    pickle.dump(segmentation_datasets,f)